<a href="https://colab.research.google.com/github/umer-ateeq/Learning-Transfromers/blob/main/GPT_2_on_tinyStories_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

init

In [ ]:
import torch
!pip install tiktoken
import tiktoken
import math

In [ ]:
# GPT_CONFIG_124M = {
#     "num_workers": 0, # usually 2-4 is safer! # This controls how many subprocesses PyTorch uses to load the data in parallel.
#     "batch_size":2, # Llama 2 7B was trained with a batch size of 1024
#     "context_length": 64//2, # for 50% data overlap!
#     "Stride":64,
#     "vocab_size": 50257,   # Vocabulary size
#     "emb_dim": 128,        # Embedding dimension
#     "n_heads": 2,         # Number of attention heads
#     "n_layers": 2,        # Number of layers
#     "drop_rate": 0.1,      # Dropout rate
#     "qkv_bias": False      # Query-key-value bias
# }

In [ ]:
# emb dim, n_head, layeer to increase size of model.
GPT_CONFIG_124M = {
    "num_batches_per_epoch" : 1000,
    "num_workers": 2, # usually 2-4 is safer! # This controls how many subprocesses PyTorch uses to load the data in parallel.
    "batch_size":64, # Llama 2 7B was trained with a batch size of 1024
    "context_length":256,    #512//2, # for 50% data overlap!
    "Stride":256,
    "vocab_size": 50257,   # Vocabulary size
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # 768/64=12 Number of attention heads  # emb_dim / batch_size =
    "n_layers": 8,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}
# print(" 1M ..train==eval \n MQA+RoPE,\n distri train,\n andrej4h(optimisations),\n MoE, \n MLA+MoE(deepseek), \n Flash attention triton")

Data Prep

In [ ]:
!pip install --upgrade datasets huggingface_hub fsspec
import datasets
import fsspec
import huggingface_hub
print("Datasets version:", datasets.__version__)
print("FSSpec version:", fsspec.__version__)
print("Huggingface_hub version:", huggingface_hub.__version__)

from datasets import load_dataset

# Clear the default cache
import shutil
import os
cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Default cache cleared!")

# Load the dataset with a custom cache directory
ds = load_dataset("roneneldan/TinyStories", cache_dir="/content/hf_datasets_cache")
print(ds)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

...

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


In [ ]:

import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2")

def process(example):
    ids = enc.encode_ordinary(example['text']) # encode_ordinary ignores any special tokens
    out = {'ids': ids, 'len': len(ids)}
    return out

if not os.path.exists("train.bin"):
    tokenized = ds.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=8,
        )
    # concatenate all the ids in each dataset into one large file we can use for training
    for split, dset in tokenized.items():
        arr_len = np.sum(dset['len'], dtype=np.uint64)
        filename = f'{split}.bin'
        dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
        total_batches = 1024

        idx = 0
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
            # Batch together samples for faster write
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
            arr_batch = np.concatenate(batch['ids'])
            # Write into mmap
            arr[idx : idx + len(arr_batch)] = arr_batch
            idx += len(arr_batch)
        arr.flush()


tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

writing train.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

writing validation.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
block_size  = GPT_CONFIG_124M['context_length']
batch_size = GPT_CONFIG_124M['batch_size']
# Some functions from https://github.com/karpathy/nanoGPT/blob/master/train.py with slight modifications
def get_batch(split):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap('train.bin', dtype=np.uint16, mode='r')
    else: # '/content/drive/MyDrive/tokenized/validation.bin',
        data = np.memmap('validation.bin', dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if torch.cuda.is_available() == True:
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

Model(MHA,ff,ln,block,gpt, init)

In [ ]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0),  "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask",torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            nn.ReLU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
print(device)

134.07744 M parameters
cuda


Generate text Function

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
# if it exceeds the supported context size then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:] # CLEVER_INDEXING == if list is [0,1,2,3] and slicing is  [-2:] then it will return [2,3]
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]
        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)
        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)
        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)
    return idx

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0] # taking the size from positional embedding shape
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad(): # dont take calc backward pass
        token_ids = generate_text_simple(model=model, idx=encoded, max_new_tokens=50, context_size=context_size) # takes encoded text to generate new future text upto max_new_tokens
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()



start_context = "Every effort moves you"
token_ids = generate_text_simple(model=model,idx=text_to_token_ids(start_context, tokenizer).to(device),max_new_tokens=20,context_size=GPT_CONFIG_124M["context_length"])
token_ids.to("cuda")
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


Output text:
 Every effort moves you Construction fiance usually Cla crop Milkyctuaryaltdose Bere Loud summed()oscopicReader dopingoustic Afghan inh inserting


In [ ]:
# def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

#     # For-loop is the same as before: Get logits, and only focus on last time step
#     for _ in range(max_new_tokens):
#         idx_cond = idx[:, -context_size:]
#         with torch.no_grad():
#             logits = model(idx_cond)
#         logits = logits[:, -1, :]

#         # New: Filter logits with top_k sampling
#         if top_k is not None:
#             # Keep only top_k values
#             top_logits, _ = torch.topk(logits, top_k)
#             min_val = top_logits[:, -1]
#             logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

#         # New: Apply temperature scaling
#         if temperature > 0.0:
#             logits = logits / temperature

#             # Apply softmax to get probabilities
#             probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

#             # Sample from the distribution
#             idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

#         # Otherwise same as before: get idx of the vocab entry with the highest logits value
#         else:
#             idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

#         if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
#             break

#         # Same as before: append sampled index to the running sequence
#         idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

#     return idx

# torch.manual_seed(123)


# # token_ids = generate(model=model, idx=text_to_token_ids(start_context, tokenizer).to("cuda"),
# #                       max_new_tokens=20, context_size=GPT_CONFIG_124M["context_length"], temperature=0.5, top_k=None, eos_id=None)

# # print("Output text:\n", token_ids_to_text(token_ids, tokenizer))





# token_ids = generate(model=model,idx=text_to_token_ids("Every effort moves you", tokenizer).to("cuda"),
#                       max_new_tokens=15,context_size=GPT_CONFIG_124M["context_length"],top_k=25,temperature=1.4)
# # token_ids
# print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Evaluation

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    vocab_size = GPT_CONFIG_124M['vocab_size']
    batch_size, seq_len = input_batch.shape
    tbatch_size, tseq_len = target_batch.shape
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.view(batch_size * seq_len, vocab_size), target_batch.view(tbatch_size * tseq_len)) #logits: shape [2, 3, 4]  logits.flatten(0, 1):shape[6, 4]
    return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches # main formula of loss loader is this: total_loss / num_batches, everything else is a error handling

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

Optimizer

In [ ]:
learning_rate = 1e-4 #more stable training, earlier 1e-4
max_iters = 20000 #increase from 25000
warmup_steps = 1000 #smoother initial train, earlier 100
min_lr = 5e-4 #lower rate, earlier 5e-4
eval_iters = 500 # increased from 100
batch_size = 32 # changed from 16, better gradient estimate
block_size = 128 #changed from 64, capture longer range dependencies

gradient_accumulation_steps = 32 # reduced from 50


from torch.optim.lr_scheduler import LinearLR,SequentialLR, CosineAnnealingLR

##PUT IN WEIGHT DECAY, CHANGED BETA2 to 0.95
optimizer =  torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.95), weight_decay=0.1, eps=1e-9) #weight decay for regularization
scheduler_warmup = LinearLR(optimizer, total_iters = warmup_steps) #Implement linear warmup
scheduler_decay = CosineAnnealingLR(optimizer,T_max = max_iters - warmup_steps, eta_min = min_lr) #Implement lr decay
scheduler = SequentialLR(optimizer, schedulers=[scheduler_warmup, scheduler_decay], milestones=[warmup_steps]) #Switching from warmup to decay
# https://stackoverflow.com/questions/72534859/is-gradscaler-necessary-with-mixed-precision-training-with-pytorch
# scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

Training(train func, train, visualise)

In [ ]:
# # import torch
# import numpy as np
# import math

# def train_model_simple(model, optimizer, device, num_epochs, num_batches_per_epoch, eval_freq, eval_iter, start_context, tokenizer):
#     # Initialize lists to track losses and tokens seen
#     train_losses, val_losses, track_tokens_seen = [], [], []
#     tokens_seen, global_step = 0, -1

#     # Main training loop
#     for epoch in range(num_epochs):
#         model.train()  # Set model to training mode

#         for _ in range(num_batches_per_epoch):
#             input_batch, target_batch = get_batch("train")
#             # optimizer.zero_grad()  # Reset loss gradients
#             loss = calc_loss_batch(input_batch, target_batch, model, device)

#             loss = loss / gradient_accumulation_steps ######

#             loss.backward()  # Calculate loss gradients
#             if (batch_idx + 1) % gradient_accumulation_steps == 0:
#               torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#               optimizer.step()  # Update model weights
#               optimizer.zero_grad(set_to_none=True)
#               scheduler.step()
#               tokens_seen += input_batch.numel()
#               global_step += 1

#             # Optional evaluation step
#             if global_step % eval_freq == 0:
#                 train_loss = estimate_loss(model, "train", eval_iter, device)
#                 val_loss = estimate_loss(model, "val", eval_iter, device)
#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 track_tokens_seen.append(tokens_seen)
#                 current_lr = optimizer.param_groups[0]['lr']
#                 print(f"Ep {epoch+1} (Step {global_step:06d}): "
#                       f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}, "
#                       f"Perplexity {math.exp(val_loss):.3f}",
#                       f"LR{current_lr:.3f}")

#         # Print a sample text after each epoch
#         generate_and_print_sample(model, tokenizer, device, start_context)

#     return train_losses, val_losses, track_tokens_seen

def estimate_loss(model, split, eval_iter, device):
    losses = []
    model.eval()
    with torch.no_grad():
        for _ in range(eval_iter):
            X, y = get_batch(split)
            loss = calc_loss_batch(X, y, model, device)
            losses.append(loss.item())
    model.train()
    return np.mean(losses)

In [ ]:
######################  lr, clipping, Mixed precision, gradient accumulation   #############################

from torch.cuda.amp import autocast, GradScaler

def train_model_simple(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    num_batches_per_epoch,
    eval_freq,
    eval_iter,
    start_context,
    tokenizer,
    gradient_accumulation_steps=1,
    precision_dtype=torch.float16
):
    # Track metrics
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Use always-on mixed precision (GPU-only)
    scaler = GradScaler()
    ctx = autocast(device_type='cuda', dtype=precision_dtype)

    model.train()
    optimizer.zero_grad(set_to_none=True)

    for epoch in range(num_epochs):
        for batch_idx in range(num_batches_per_epoch):
            # Get batch and move to GPU
            input_batch, target_batch = get_batch("train")
            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)

            # Forward pass in mixed precision
            with ctx:
                loss = calc_loss_batch(input_batch, target_batch, model, device)

            # Normalize loss for accumulation
            loss = loss / gradient_accumulation_steps # each call is 1/8.
            scaler.scale(loss).backward()

            # Gradient accumulation step
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer) # unscale grads back to float32 from float16
                scaler.update() # 	Learns whether to increase or decrease scale for next step
                optimizer.zero_grad(set_to_none=True)
                scheduler.step() # hey, update the learning rate according to the current step or epoch

                tokens_seen += input_batch.numel() * gradient_accumulation_steps
                global_step += 1

                # Evaluation checkpoint
                if global_step % eval_freq == 0:
                    train_loss = estimate_loss(model, "train", eval_iter, device)
                    val_loss = estimate_loss(model, "val", eval_iter, device)
                    train_losses.append(train_loss)
                    val_losses.append(val_loss)
                    track_tokens_seen.append(tokens_seen)

                    current_lr = optimizer.param_groups[0]['lr'] # for printing but can be explored!!!
                    print(
                        f"Ep {epoch+1} (Step {global_step:06d}): "
                        f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}, "
                        f"Perplexity {math.exp(val_loss):.3f}, LR {current_lr:.6f}"
                    )

        # Sample generation after each epoch
        generate_and_print_sample(model, tokenizer, device, start_context)

    return train_losses, val_losses, track_tokens_seen


In [ ]:
# ###############  GRADIENT ACCUMULATION VERSION!!!!! ###################

# import numpy as np
# import math

# def train_model_simple(model, optimizer, scheduler, device, num_epochs, num_batches_per_epoch,
#                        eval_freq, eval_iter, start_context, tokenizer, gradient_accumulation_steps=1):
#     train_losses, val_losses, track_tokens_seen = [], [], []
#     tokens_seen, global_step = 0, -1

#     model.train()
#     optimizer.zero_grad(set_to_none=True)  # init once before training

#     for epoch in range(num_epochs):
#         for batch_idx in range(num_batches_per_epoch):
#             input_batch, target_batch = get_batch("train")
#             input_batch, target_batch = input_batch.to(device), target_batch.to(device)

#             loss = calc_loss_batch(input_batch, target_batch, model, device)
#             loss = loss / gradient_accumulation_steps
#             loss.backward()

#             # Gradient accumulation
#             if (batch_idx + 1) % gradient_accumulation_steps == 0:
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#                 optimizer.step()
#                 optimizer.zero_grad(set_to_none=True)
#                 scheduler.step()

#                 tokens_seen += input_batch.numel() * gradient_accumulation_steps
#                 global_step += 1

#                 # Optional evaluation
#                 if global_step % eval_freq == 0:
#                     train_loss = estimate_loss(model, "train", eval_iter, device)
#                     val_loss = estimate_loss(model, "val", eval_iter, device)
#                     train_losses.append(train_loss)
#                     val_losses.append(val_loss)
#                     track_tokens_seen.append(tokens_seen)
#                     current_lr = optimizer.param_groups[0]['lr']
#                     print(f"Ep {epoch+1} (Step {global_step:06d}): "
#                           f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}, "
#                           f"Perplexity {math.exp(val_loss):.3f}, LR {current_lr:.6f}")

#         # Generate sample text after each epoch
#         generate_and_print_sample(model, tokenizer, device, start_context)

#     return train_losses, val_losses, track_tokens_seen


In [ ]:
# def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_freq, eval_iter, start_context, tokenizer):
#     # Initialize lists to track losses and tokens seen
#     train_losses, val_losses, track_tokens_seen = [], [], []
#     tokens_seen, global_step = 0, -1

#     # Main training loop
#     for epoch in range(num_epochs):
#         model.train()  # Set model to training mode

#         for input_batch, target_batch in get_batch("train"):
#             optimizer.zero_grad() # Reset loss gradients from previous batch iteration
#             loss = calc_loss_batch(input_batch, target_batch, model, device)
#             loss.backward() # Calculate loss gradients
#             optimizer.step() # Update model weights using loss gradients
#             tokens_seen += input_batch.numel() ### if shape(3,3) .numel will return 3*3 = 9
#             global_step += 1

#             # Optional evaluation step
#             if global_step % eval_freq == 0:
#                 train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 track_tokens_seen.append(tokens_seen)
#                 print(f"Ep {epoch+1} (Step {global_step:06d}): "
#                       f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f} "
#                       f"perplexity {math.exp(val_loss):.3f}"
#                       )

#         # Print a sample text after each epoch
#         generate_and_print_sample(model, tokenizer, device, start_context)

#     return train_losses, val_losses, track_tokens_seen

In [ ]:
# print(model)

In [ ]:
# GPT_CONFIG_124M["num_batches_per_epoch"]

In [ ]:
# Example usage
  # Start with 1,000 batches
num_epochs = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")
optim = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

train_losses, val_losses, tokens_seen = train_model_simple(
    model=model,  # Your model
    optimizer=optim,  # Your optimizer
    device=device,
    num_epochs=num_epochs,
    num_batches_per_epoch=GPT_CONFIG_124M["num_batches_per_epoch"],
    eval_freq=100,
    eval_iter=10,
    start_context="Once upon a time",
    tokenizer=tokenizer
)

TypeError: train_model_simple() missing 1 required positional argument: 'scheduler'

In [ ]:
# # # Uncomment the following code to calculate the execution time
# import time
# start_time = time.time()

# num_epochs = 10
# # train_losses, val_losses, tokens_seen = train_model_simple(
# #     model, train_loader, val_loader, optim, device,
# #     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
# #     start_context="Every effort moves you", tokenizer=tokenizer
# # )
# train_losses, val_losses, tokens_seen = train_model_simple(
#     model=model,  # Your model
#     optimizer=optim,  # Your optimizer
#     device=device,
#     num_epochs=num_epochs,
#     num_batches_per_epoch=GPT_CONFIG_124M["num_batches_per_epoch"],
#     eval_freq=100,
#     eval_iter=10,
#     start_context="Once upon a time",
#     tokenizer=tokenizer
# )
# # Uncomment the following code to show the execution time
# end_time = time.time()
# execution_time_minutes = (end_time - start_time) / 60
# print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
# # # Uncomment the following code to calculate the execution time
# import time
# start_time = time.time()

# num_epochs = 100
# # train_losses, val_losses, tokens_seen = train_model_simple(
# #     model, train_loader, val_loader, optim, device,
# #     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
# #     start_context="Every effort moves you", tokenizer=tokenizer
# # )
# train_losses, val_losses, tokens_seen = train_model_simple(
#     model=model,  # Your model
#     optimizer=optim,  # Your optimizer
#     device=device,
#     num_epochs=num_epochs,
#     num_batches_per_epoch=GPT_CONFIG_124M["num_batches_per_epoch"],
#     eval_freq=100,
#     eval_iter=10,
#     start_context="Once upon a time",
#     tokenizer=tokenizer
# )
# # Uncomment the following code to show the execution time
# end_time = time.time()
# execution_time_minutes = (end_time - start_time) / 60
# print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
# import torch
# import seaborn as sns
# import matplotlib.pyplot as plt

# def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
#     plt.figure(figsize=(5, 3))

#     # Plot training and validation loss against epochs
#     sns.lineplot(x=epochs_seen, y=train_losses, label="Training loss")
#     sns.lineplot(x=epochs_seen, y=val_losses, linestyle="-.", label="Validation loss")

#     plt.xlabel("Epochs")
#     plt.ylabel("Loss")
#     # plt.legend(loc="upper right")

#     # Create a second x-axis for tokens seen
#     plt.twiny()
#     sns.lineplot(x=tokens_seen, y=train_losses, alpha=0)  # Invisible plot for aligning ticks
#     plt.xlabel("Tokens seen")

#     # plt.tight_layout()  # Adjust layout to make room
#     plt.savefig("loss-plot.pdf")
#     plt.show()

# # Example usage (ensure these tensors are defined in your script)
# epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
# plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)


Loading and saving model weights in PyTorch

In [ ]:
# torch.save(model.state_dict(), "model.pth")
torch.save(model.state_dict(), "tinystories211epoch.pth")



# # - Then we can load the model weights into a new `GPTModel` model instance as follows:
# model = GPTModel(GPT_CONFIG_124M)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))

# model.eval();

### adaptive optimizers store additional params for each model weight,so it makes sense to save them as well in case we plan to continue pretraining

# torch.save({
#     "model_state_dict": model.state_dict(),
#     "optimizer_state_dict": optim.state_dict(),
#     },
#     "model_and_optimizer.pth"
# )

# checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)

# model = GPTModel(GPT_CONFIG_124M)
# model.to(device)
# model.load_state_dict(checkpoint["model_state_dict"])

# optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# torch.manual_seed(123)
# model.to(device)




In [ ]:
from google.colab import files
files.download("tinystories211epoch.pth")

In [ ]:
# # Uncomment the following code to calculate the execution time
import time
start_time = time.time()

num_epochs = 100
# train_losses, val_losses, tokens_seen = train_model_simple(
#     model, train_loader, val_loader, optim, device,
#     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
#     start_context="Every effort moves you", tokenizer=tokenizer
# )
train_losses, val_losses, tokens_seen = train_model_simple(
    model=model,  # Your model
    optimizer=optim,  # Your optimizer
    device=device,
    num_epochs=num_epochs,
    num_batches_per_epoch=GPT_CONFIG_124M["num_batches_per_epoch"],
    eval_freq=100,
    eval_iter=10,
    start_context="Once upon a time",
    tokenizer=tokenizer
)
# Uncomment the following code to show the execution time
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    plt.figure(figsize=(5, 3))

    # Plot training and validation loss against epochs
    sns.lineplot(x=epochs_seen, y=train_losses, label="Training loss")
    sns.lineplot(x=epochs_seen, y=val_losses, linestyle="-.", label="Validation loss")

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    # plt.legend(loc="upper right")

    # Create a second x-axis for tokens seen
    plt.twiny()
    sns.lineplot(x=tokens_seen, y=train_losses, alpha=0)  # Invisible plot for aligning ticks
    plt.xlabel("Tokens seen")

    # plt.tight_layout()  # Adjust layout to make room
    plt.savefig("loss-plot.pdf")
    plt.show()

# Example usage (ensure these tensors are defined in your script)
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)


In [ ]:
torch.save(model.state_dict(), "tinystories211epoch.pth")
from google.colab import files
files.download("tinystories211epoch.pth")

In [ ]:
# # Uncomment the following code to calculate the execution time
import time
start_time = time.time()

num_epochs = 100
# train_losses, val_losses, tokens_seen = train_model_simple(
#     model, train_loader, val_loader, optim, device,
#     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
#     start_context="Every effort moves you", tokenizer=tokenizer
# )
train_losses, val_losses, tokens_seen = train_model_simple(
    model=model,  # Your model
    optimizer=optim,  # Your optimizer
    device=device,
    num_epochs=num_epochs,
    num_batches_per_epoch=GPT_CONFIG_124M["num_batches_per_epoch"],
    eval_freq=100,
    eval_iter=10,
    start_context="Once upon a time",
    tokenizer=tokenizer
)
# Uncomment the following code to show the execution time
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    plt.figure(figsize=(5, 3))

    # Plot training and validation loss against epochs
    sns.lineplot(x=epochs_seen, y=train_losses, label="Training loss")
    sns.lineplot(x=epochs_seen, y=val_losses, linestyle="-.", label="Validation loss")

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    # plt.legend(loc="upper right")

    # Create a second x-axis for tokens seen
    plt.twiny()
    sns.lineplot(x=tokens_seen, y=train_losses, alpha=0)  # Invisible plot for aligning ticks
    plt.xlabel("Tokens seen")

    # plt.tight_layout()  # Adjust layout to make room
    plt.savefig("loss-plot.pdf")
    plt.show()

# Example usage (ensure these tensors are defined in your script)
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)


In [ ]:
torch.save(model.state_dict(), "tinystories311epoch.pth")
from google.colab import files
files.download("tinystories311epoch.pth")

RESTART Training!!!

In [ ]:
checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)

model = GPTModel(GPT_CONFIG_124M)
model.to(device)
model.load_state_dict(checkpoint["model_state_dict"])

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

torch.manual_seed(123)
model.to(device)

num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer)

torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optim.state_dict(),
    },
    "model_and_optimizer.pth"
)
# from google.colab import files
# files.download("model_and_optimizer.pth")


In [ ]:
# torch.save(model.state_dict(), "model.pth")

# checkpoint = torch.load("model.pth", weights_only=True)

# model = GPTModel(GPT_CONFIG_124M)
# model.to(device)
# model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))

# optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
# # optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# torch.manual_seed(123)
# model.to(device)

# num_epochs = 1
# train_losses, val_losses, tokens_seen = train_model_simple(
#     model, train_loader, val_loader, optimizer, device,
#     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
#     start_context="Every effort moves you", tokenizer=tokenizer)